In [1]:
import pandas as pd
from evds import evdsAPI
import warnings
import pandas as pd
from thefuzz import fuzz
from thefuzz import process
import geopandas as gp
from matplotlib import pyplot as plt

C:\ProgramData\Anaconda3\lib\site-packages\thefuzz\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
warnings.filterwarnings("ignore", category=Warning)
warnings.filterwarnings("ignore", category=FutureWarning)

evds=evdsAPI('***********')
a=evds.get_series('bie_hkfe')

a

EVDSConnectionError: Connection error, please check your API Key or request. Url:https://evds2.tcmb.gov.tr/service/evds/categories/key=***********&type=json

In [ ]:
# Calculate the annual percentage change for Residential Property Price Index index (KFE) data
kfe=evds.get_data(list(a.SERIE_CODE),startdate=a["START_DATE"].unique()[0],enddate="15-08-2040")
kfe.iloc[:,1:]=kfe.iloc[:,1:].pct_change(12)*100
a["SERIE_CODE"]=a["SERIE_CODE"].str.replace(".","_")

# Prepare data for geospatial mapping
kfe=(pd.melt(kfe, id_vars=["Tarih"], var_name="SERIE_CODE")
    .merge(a[["SERIE_CODE","SERIE_NAME"]], on="SERIE_CODE")
    .assign(Tarih=lambda x: pd.to_datetime(x["Tarih"],infer_datetime_format=True))
    .query('Tarih==Tarih.max()')
    .drop(["SERIE_CODE"],axis=1)
    .query('SERIE_NAME!="Konut Fiyat Endeksi (KFE)"')
    .assign(KOD=lambda x:x["SERIE_NAME"].str.split("(",expand=True)[0])
    .assign(City=lambda x:x["SERIE_NAME"].str.split("(",expand=True)[1])
    .assign(City=lambda x: x["City"].str.replace(")","")))
kfe_il = kfe["City"].str.split(",", expand=True).ffill(axis=1)
kfe = (pd.concat([kfe, kfe_il], axis=1)
       .rename(columns={"value": "value_"})
       .drop(["SERIE_NAME", "Tarih", "KOD"], axis=1))

# Create a dictionary and DataFrame for mapping
il_sozluk = {}
for index, row in kfe.iterrows():
    for i in range(0, 6):
        il_sozluk[row[i]] = row["value_"]
kfe = pd.DataFrame(il_sozluk.items(), columns=['City', "KFE"])
kfe["City"] = kfe["City"].str.strip()
kfe

In [ ]:



turkish_cities_geopoints = (gp.read_file(f'shape_files/TUR_ADM1.shp')
                           .assign(Name=lambda x:x["Name"].str.strip())
                           .assign(Name=lambda x:x["Name"].replace('KIR?EH?R', 'Kırşehir'))
                           .assign(Name=lambda x:x["Name"].replace('A?RI', 'Ağrı'))
                           .assign(City=lambda x:x["Name"].apply(lambda x: process.extractOne(x, kfe["City"], scorer=fuzz.ratio)[0]))
                           .assign(ratio=lambda x:x["Name"].apply(lambda x: process.extractOne(x, kfe["City"], scorer=fuzz.ratio)[1]))
                           .merge(kfe, on ="City")
                           .drop(["City","ratio"],axis=1))

turkish_cities_geopoints



In [ ]:
#Graph  phase
title = 'Turkish RPI (Yearly Change, %)'
col="KFE"
vmin = turkish_cities_geopoints[col].min()
vmax = turkish_cities_geopoints[col].max()
cmap = 'Reds'

fig, ax = plt.subplots(1, figsize=(20, 8))
ax.axis('off')
turkish_cities_geopoints.plot(column=col, ax=ax, edgecolor='0.8', linewidth=1, cmap=cmap)
ax.set_title(title, fontdict={'fontsize': '20', 'fontweight': '3'})
sm = plt.cm.ScalarMappable(norm=plt.Normalize(vmin=vmin, vmax=vmax), cmap=cmap)
cbaxes = fig.add_axes([0.15, 0.25, 0.01, 0.4])
cbar = fig.colorbar(sm, cax=cbaxes)
plt.savefig('result_files/residential_price.png')
plt.savefig('result_files/residential_price.pdf')
writer = pd.ExcelWriter('result_files//residential_price.xlsx', engine='xlsxwriter')


In [ ]:

# Excel Phase
turkish_cities_geopoints.to_excel(writer, sheet_name='Sheet1')

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet = writer.sheets['Sheet1']
worksheet.insert_image('D3', 'residential_price.png')
writer.save()